In [1]:
import numpy as np
import math

In [3]:
# Parametros extrinsecos
phi = np.pi / 3.0 # Angulo de rotacion con respecto al eje x
R = np.zeros((3, 3)) # Matriz de rotacion
R[0][0] = 1.0
R[1][1] = R[2][2] = math.cos(phi)
R[2][1] = math.sin(phi)
R[1][2] = -R[2][1]
t = np.array([3.0, 2.0, 1.0]) # Vector de traslacion

In [4]:
# Parametros intrinsecos
theta = 0.9 * 0.5 * np.pi
u = 5.0
v = 7.0
alpha_u = 10.0
alpha_v = 15.0

In [38]:
def armar_matriz_P(theta, u, v, alpha_u, alpha_v, R, t):
    P = np.ones((3, 4))
    P[0,0:3] = alpha_u*R[:,0].T - (alpha_u/math.tan(theta))*R[:,1].T + u*R[:,2].T
    P[0][3] = alpha_u*t[0] - (alpha_u/math.tan(theta))*t[1] + u*t[2]
    P[1,0:3] = (alpha_v/math.sin(theta))*R[:,1].T + v*R[:,2].T
    P[1][3] = (alpha_v/math.sin(theta))*t[1] + v*t[2]
    P[2,0:3] = R[:,2].T
    P[2][3] = t[2]
    return P

In [39]:
P = armar_matriz_P(theta, u, v, alpha_u, alpha_v, R, t)

In [42]:
def descomponer_P(P):
    lamda = np.linalg.norm(P[2,0:3])
    P = P/lamda
    r3 = P[2,0:3]
    v = np.dot(r3, P[1,0:3])
    g = np.linalg.norm(P[1,0:3] - v*r3)
    r2 = (P[1,0:3] - v*r3)/g
    t = np.zeros(3)
    t[2] = P[2][3]
    t[1] = (P[1][3] - v*t[2])/g
    u = np.dot(r3, P[0,0:3])
    h = np.dot(r2, P[0,0:3])
    alpha_u = np.linalg.norm(P[0,0:3] - u*r3 - h*r2)
    r1 = (P[0,0:3] - u*r3 - h*r2)/alpha_u
    theta = math.atan(alpha_u/h)
    alpha_v = g*math.sin(theta)
    t[0] = (P[0][3] - h*t[1] - u*t[2])/alpha_u
    R = np.array([r1, r2, r3]).T
    return [theta, u, v, alpha_u, alpha_v, R, t]

In [43]:
parametros = descomponer_P(P)

In [44]:
parametros

[-1.413716694115407,
 5.0,
 7.000000000000001,
 10.0,
 -15.0,
 array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 6.66133815e-17,  5.00000000e-01, -8.66025404e-01],
        [ 6.66133815e-17,  8.66025404e-01,  5.00000000e-01]]),
 array([3., 2., 1.])]

La interpretacion geometrica de las dos descomposiciones de P esta en los signos de las normas.